In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSM_FILE = "C:\Chaitanya\Jupyter\Data_Wrangling\Project_DataWrangling\houston_texas.osm"
SAMPLE_FILE = "C:\Chaitanya\Jupyter\Data_Wrangling\Project_DataWrangling\houston_texas_sample.osm"

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
#pincode_re = re.compile(r'\w*\d{5}', re.IGNORECASE)
#street_EWNS_re = re.compile(r'^([a-z]){1}\.?(\s)+', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "East", "West", "North", "South", "Freeway", "Highway", "Circle", "Park"]

mapping = { "St": "Street", "St.": "Street", "street": "Street", "Ave": "Avenue", "Rd.": 'Road', "Rd": "Road",\
           "Blvd": "Boulevard", "Blvd.": "Boulevard", "Dr": "Drive", "Fwy": "Freeway", "Frwy": "Freeway", "Hwy": "Highway",\
           "N": "North", "N.": "North", "W": "West", "W.": "West", "E": "East", "E.": "East", "S": "South", "S.": "South",\
           "Ln": "Lane", "Farm-to-Market Road 1774": "FM 1774"}


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)



def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types




def update_name(name, mapping):
    
    better_name = ""
    m = street_type_re.search(name)
    if m:
        st_type = m.group()
        if (st_type in mapping.keys()):
            better_name = re.sub(st_type, mapping[st_type], name)
        else:
            better_name = name
                      
    return better_name


def test():
    st_types = audit(SAMPLE_FILE)
    pprint.pprint(dict(st_types))
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name1 = truncate_extension(name)
            better_name2 = update_name(better_name1, mapping)
            print name, "=>", better_name2
            


if __name__ == '__main__':
    test()

{'1/2': set(['Avenue P 1/2']),
 '10': set(['I 10']),
 '1093': set(['FM 1093']),
 '1142': set(['Lake Woodlands Drive #1142']),
 '146': set(['Texas 146']),
 '1774': set(['Farm-to-Market Road 1774']),
 '185': set(['I-45 South, Suite 185']),
 '1960': set(['FM 1960']),
 '249': set(['TX 249']),
 '2920': set(['FM 2920']),
 '521': set(['FM 521']),
 '6': set(['TX 6']),
 '646': set(['FM 646']),
 '77598': set(['Gulf Freeway, Webster, Texas 77598']),
 '90': set(['US 90']),
 '925': set(['Katy Freeway Suite 925']),
 'Americas': set(['Avenida de Las Americas']),
 'B': set(['Richmond Ave, Ste B']),
 'Blvd': set(['Seawall Blvd', 'Sweetwater Blvd']),
 'Crossing': set(['Woodland Field Crossing']),
 'Dr': set(['Memorial Dr', 'Park Row Dr', 'Post Oak Place Dr']),
 'Driscoll': set(['Driscoll']),
 'F': set(['Avenue F']),
 'F-3': set(['Memorial Drive, Suite F-3']),
 'Frwy': set(['East Frwy']),
 'Fwy': set(['Gulf Fwy']),
 'G': set(['W 34th St #G']),
 'Graustark': set(['Graustark']),
 'I-45': set(['18484 I-45']

NameError: global name 'truncate_extension' is not defined